In [3]:
#! /usr/bin/python3
# -*- coding: utf-8 -*-


from lxml import html
import requests


r = requests.get('http://m.wikitree.co.kr/main/news_view.php?id=217101')
root = html.document_fromstring(r.text)
string = '\n'.join(root.xpath('//div[@id="descs"]/div//text()'))
# string = '\n'.join(root.xpath('//div[@id="descs"]/div//text()'))


In [7]:
for i in range(1, 21):
    tmp = str(string.split('%s.' % i, 1))
    print (tmp.decode('euc-kr').encode('utf-8'))


AttributeError: 'str' object has no attribute 'decode'

In [134]:
# -*- coding: utf-8 -*-

import requests
import re
from bs4 import BeautifulSoup
import json
import csv
import numpy as np
from lxml import html
import requests
import json

APIKEY = '**********' # 이 곳에 다음 API 키를 입력 (http://developer.naver.com/wiki/pages/OpenAPI)
MAPAPI = 'http://apis.daum.net/local/geo/addr2coord?apikey=%s&q=%s&output=json' # API 결과물을 json파일 형태로 받는다

def get_latlon(query):
    # query값으로 주소 형태의 데이터가 들어오면 다음 API를 통해서 위도, 경도 값을 반환
    resp = requests.get(MAPAPI % (APIKEY, query))
    try:
        # json형태의 API 결과물중에서 위도, 경도에 해당하는 결과값만을 호출
        lat = json.loads(resp.text)['channel']['item'][0]['lat']
        lng = json.loads(resp.text)['channel']['item'][0]['lng']
        return (lat, lng)
    except:
        # 위도, 경도 값을 얻을 수 없는 주소값이 존재
        # 이러한 경우 값을 0으로 받고 추후 수기로 위도, 경도 값을 추가
        return (0, 0)

def prep(category, name, address):
    # 크롤링한 데이터중 category, name, address > 위도, 경도 값을 반환
    name = name
    lat, lon = get_latlon(address)

    return {
        'name': name,
        'category': category,
        'lat': lat, 'lon': lon,
    }


{'name': '가온', 'category': '한식', 'lat': 37.5742692547, 'lon': 126.9854105474}


In [135]:
# 미쉘린 가이드 서울 홈페이지에 올라온 데이터를 크롤링하는 코드
# 홈페이지의 데이터의 정보 중 category, name, address 값을 가지고 온다.
items = []
for num in range(1, 18):
    urls = ['https://guide.michelin.co.kr/ko/restaurant/page/{}'.format(num)]
    for url in urls:
        res = requests.get(url)
        content = res.content
        soup = BeautifulSoup(content, "html.parser")
        restaurants = soup.find_all('article', attrs ={'class' :"restaurant-list-wrap"})
        for restaurant in restaurants:
            category = restaurant.find('span', attrs={'class':"restaurant-list-category"}).get_text()
            name = restaurant.find(attrs={'class':"restaurant-list-title"}).get_text()
            address = restaurant.find('p', attrs={'class':"ellipsis"}).get_text().strip()
            items.append([category, name, address])

# 크롤링한 결과를 CSV파일로 저장하기 위한 형태로 정의
data = [prep(i[0],i[1],i[2]) for i in items[:]]

# 식당 이름, 위도, 경도 값을 CSV파일 형태로 저장
with open('places.csv', 'w') as f:
    f.write('name,lat,lon\n')
    for d in data:
        f.write('%(name)s,%(lat)s,%(lon)s\n' % d)

[{'name': '가온', 'category': '한식', 'lat': 37.5226374252, 'lon': 127.0360593358}, {'name': '갈리나 데이지', 'category': '이탈리안', 'lat': 37.5798707677, 'lon': 126.9709547615}, {'name': '개성만두 궁', 'category': '만두', 'lat': 37.5742692547, 'lon': 126.9854105474}, {'name': '고려 삼계탕', 'category': '삼계탕', 'lat': 0, 'lon': 0}, {'name': '고봉 삼계탕', 'category': '삼계탕', 'lat': 37.5647329166, 'lon': 126.9838594761}]
